
# Space

In [1]:
import os
import sys 
import logging
import pandas as pd 
from pprint import pprint 

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
# pprint(SPACE)

# Available Packages
import pandas as pd
from datetime import datetime 

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')

import datasets
from recfldtkn.loadtools import load_ds_rec_and_info
from recfldtkn.configfn import load_cohort_args, load_record_args
from config_observer.CKPD import ckpd_to_CkpdObsConfig


cohort_config = load_cohort_args(recfldtkn_config_path, SPACE)
cohort_config['ckpd_to_CkpdObsConfig'] = ckpd_to_CkpdObsConfig
cohort_config['ObsDTName'] = 'ObsDT'
cohort_config['PID_ObsDT_columns'] = [cohort_config['RootID'], cohort_config['ObsDTName']]
print(cohort_config)

g:\Shared drives\CDHAI-WellDoc\2024-WellDocTest-SPACE\_WellDoc-AI-CGMGPT-WorkSpace
{'CohortInfo': {'RawData2022_CGM': {'cohort_label': 1, 'cohort_name': 'RawData2022_CGM', 'FolderPath': '../_Data/0-Data_Raw/RawData2022_CGM/'}, 'RawData2023_CVSTDCAug': {'cohort_label': 2, 'cohort_name': 'RawData2023_CVSTDCAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/'}, 'RawData2023_CVSDeRxAug': {'cohort_label': 3, 'cohort_name': 'RawData2023_CVSDeRxAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSDeRxAug/'}}, 'RawRootID': 'PatientID', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_to_RFT_GROUP_SIZE': {'CGM5Min': 100, 'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'CGM5Min': 100, 'BGManual': 100, 'Default': 10000}, 'RecName_to_RFT_useb

# 1. DsCase

In [2]:
# 0. ************ RFT config ************
RecName_to_dsRec, RecName_to_dsRecInfo = {}, {}
cohort_label_list = [1, 2, 3]

# 1. ************ Case Trigger config ************
TriggerCaseMethod = 'CGM5MinEntry'


# 2. ************ InputCaseSetName ************
# option 1
# InputCaseSetName = 'C1.2.3-CGM5MinEntry'
# InputCaseSetName = 'sftcgmbf24haf2h-rs42-ds0.1-out0.1tstail0.1vd0.1'
InputCaseSetName = None 
# InputCaseSetName = 'C123-CGM-tagfltBfAfCGMinfo-R42ds10'

In [3]:
from recfldtkn.loadtools import fetch_trigger_tools

Trigger_Tools = fetch_trigger_tools(TriggerCaseMethod, SPACE)
case_id_columns = Trigger_Tools['case_id_columns']
cohort_config['case_id_columns'] = case_id_columns
case_id_columns

['PID', 'ObsDT']

In [4]:
from recfldtkn.pipeline_case import get_ds_case_to_process

InputCaseSetName, df_case = get_ds_case_to_process(InputCaseSetName, 
                                                   cohort_label_list, 
                                                   TriggerCaseMethod, 
                                                   cohort_config, 
                                                   SPACE, 
                                                   RecName_to_dsRec, 
                                                   RecName_to_dsRecInfo)

logger.info(f'InputCaseSetName: {InputCaseSetName}')
logger.info(f'df_case shape: {df_case.shape}')

[INFO:2024-04-21 09:14:12,075:(269221877.py@11 __main__)]: InputCaseSetName: C1.2.3-CGM5MinEntry-sz21215912
[INFO:2024-04-21 09:14:12,075:(269221877.py@12 __main__)]: df_case shape: (21215912, 2)


In [5]:
df_case = df_case.sample(1000)
df_case.shape

(1000, 2)

# 2. Tagging

In [6]:
from recfldtkn.pipeline_case import process_df_tagging_tasks_in_chunks
from config_observer.QCF import cf_to_QueryCaseFeatConfig


CASE_TAGGING_PROC_CONFIG = {
    'use_CF_from_disk': False,
    'use_CO_from_disk': False,
    'start_chunk_id': 0,
    'end_chunk_id': None,
    'chunk_size': 500000,
    'save_to_pickle': False,
    'num_processors': 1
}

######################################
TagMethod_List = [ 'PttBasicWD', 'Bf1mMEDALcf', 'Bf24hCGMrn', 'Af2hCGMrn',]
######################################

if len(TagMethod_List) > 0:
    logger.info(f'df_case shape: {df_case.shape}')
    OutputCaseSetName, df_case = process_df_tagging_tasks_in_chunks(df_case, cohort_label_list, case_id_columns, 
                                                                    InputCaseSetName, 
                                                                    TagMethod_List, cf_to_QueryCaseFeatConfig, 
                                                                    cohort_config, SPACE, 
                                                                    RecName_to_dsRec, RecName_to_dsRecInfo,
                                                                    **CASE_TAGGING_PROC_CONFIG)
    logger.info(f'df_case shape: {df_case.shape}')


else:
    pass

[INFO:2024-04-21 09:14:12,790:(1224920974.py@20 __main__)]: df_case shape: (1000, 2)
[INFO:2024-04-21 09:14:12,791:(pipeline_case.py@420 recfldtkn.pipeline_case)]: chunk_id_list: range(0, 1)


[INFO:2024-04-21 09:14:12,921:(pipeline_case.py@308 recfldtkn.pipeline_case)]: ds_info is P: Dataset({
    features: ['PID', 'PatientID', 'YearOfBirth', 'ActivationDate', 'MRSegmentModifiedDateTime', 'UserTimeZone', 'UserTimeZoneOffset', 'Gender', 'MRSegmentID', 'DiseaseType'],
    num_rows: 11621
})
[INFO:2024-04-21 09:14:13,428:(loadtools.py@156 recfldtkn.loadtools)]: No such folder: ../_Data/1-Data_RFT\3-RawData2023_CVSDeRxAug\ImpMed_data ...
[INFO:2024-04-21 09:14:13,615:(loadtools.py@156 recfldtkn.loadtools)]: No such folder: ../_Data/1-Data_RFT\1-RawData2022_CGM\LessonProg_data ...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

[INFO:2024-04-21 09:14:35,771:(pipeline_case.py@462 recfldtkn.pipeline_case)]: chunk_id: 0, df_case_chunk_tagged: (1000, 22)
[INFO:2024-04-21 09:14:35,772:(1224920974.py@27 __main__)]: df_case shape: (1000, 22)


In [7]:
print(df_case.shape)

(1000, 22)


# 3. Filtering

In [8]:
FilterMethod_List = [
    'fPttBasicWD', 
    'fBf24h280CGM',
    'fAf2h24CGM',
]

In [9]:

from recfldtkn.pipeline_case import process_df_filtering_tasks

logger.info(f'-------------- (4) FilterMethod_List: {FilterMethod_List} --------------')
if len(FilterMethod_List) > 0:
    # logger.info(f'---------- before filtering: {df_case.shape} --------------')
    df_case = process_df_filtering_tasks(df_case, FilterMethod_List, SPACE)
    # logger.info(f'---------- after filtering: {df_case.shape} --------------')
    logger.info(f'df_case shape: {df_case.shape}')

[INFO:2024-04-21 09:14:35,837:(3341762444.py@3 __main__)]: -------------- (4) FilterMethod_List: ['fPttBasicWD', 'fBf24h280CGM', 'fAf2h24CGM'] --------------
[INFO:2024-04-21 09:14:35,837:(pipeline_case.py@958 recfldtkn.pipeline_case)]: FilterMethod: fPttBasicWD
[INFO:2024-04-21 09:14:35,889:(pipeline_case.py@963 recfldtkn.pipeline_case)]: before filtering: (1000, 22)
[INFO:2024-04-21 09:14:35,890:(pipeline_case.py@965 recfldtkn.pipeline_case)]: after filtering: (988, 22)
[INFO:2024-04-21 09:14:35,891:(pipeline_case.py@958 recfldtkn.pipeline_case)]: FilterMethod: fBf24h280CGM
[INFO:2024-04-21 09:14:35,938:(pipeline_case.py@963 recfldtkn.pipeline_case)]: before filtering: (988, 22)
[INFO:2024-04-21 09:14:35,940:(pipeline_case.py@965 recfldtkn.pipeline_case)]: after filtering: (787, 22)
[INFO:2024-04-21 09:14:35,940:(pipeline_case.py@958 recfldtkn.pipeline_case)]: FilterMethod: fAf2h24CGM
[INFO:2024-04-21 09:14:36,058:(pipeline_case.py@963 recfldtkn.pipeline_case)]: before filtering: (78

# Split

In [10]:
df_case # df_case_learning

,PID,ObsDT,Gender,YearOfBirth,DiseaseType,cohort,Age,Bf1mMedPres:recnum,Bf1mMedAdmin:recnum,Bf1mImpMed:recnum,...,Bf1mFood:recnum,Bf1mCarb:recnum,Bf1mExercise:recnum,Bf1mSleep:recnum,Bf1mStep:recnum,Bf1mBP:recnum,Bf1mWeigh:recnum,Bf1mLabTest:recnum,Bf24hCGMrn:recnum,Af2hCGMrn:recnum
0,1000009,2021-02-07 14:25:00,2,1975,2,C1,46,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,289.0,24.0
1,1000016,2021-08-31 11:50:00,1,1964,2,C1,57,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,289.0,24.0
2,1000018,2021-01-24 10:50:00,1,1980,1,C1,41,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,289.0,24.0
3,1000018,2021-11-19 03:10:00,1,1980,1,C1,41,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,289.0,24.0
4,1000020,2021-02-06 02:45:00,1,1983,1,C1,38,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,289.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,3000031,2023-05-15 23:40:00,2,1990,2,C3,33,0.0,0.0,0.0,...,0.0,0.0,17.0,0.0,8.0,0.0,0.0,0.0,288.0,24.0
761,3000031,2023-07-02 07:50:00,2,1990,2,C3,33,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,289.0,24.0
762,3000039,2023-07-25 06:55:00,1,1966,2,C3,57,0.0,0.0,0.0,...,114.0,66.0,70.0,0.0,33.0,0.0,0.0,0.0,281.0,24.0
763,3000040,2023-03-11 05:15:00,2,1965,2,C3,58,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,289.0,24.0


In [11]:
from recfldtkn.pipeline_case import generate_random_tags, assign_caseSplitTag_to_dsCase

##############
# RANDOM_SEED = 42
# downsample_ratio = 1 # 0.1 # 1 (don't drop any case), 0.1 (drop 90% of cases of one patient).
# out_ratio = 0 # 0.1 # hold-out patients
# test_ratio = 0.2 # 'tail0.1' #  '2023.10.15'#  # '0.1'
# valid_ratio = 0 #  0.1 
##############

SplitDict = {
    'RootID': cohort_config['RootID'],
    'ObsDT': 'ObsDT',
    'RANDOM_SEED': 42,
    'downsample_ratio': 0.1,
    'out_ratio': 0.1,
    'test_ratio': 'tail0.1',
    'valid_ratio': 0.1
}


# SplitMethod = f'rs{RANDOM_SEED}-ds{downsample_ratio}-out{out_ratio}ts{test_ratio}vd{valid_ratio}' 
# print(SplitMethod)

In [12]:
df_case = assign_caseSplitTag_to_dsCase(df_case,  **SplitDict)
df_case

,PID,ObsDT,Gender,YearOfBirth,DiseaseType,cohort,Age,Bf1mMedPres:recnum,Bf1mMedAdmin:recnum,Bf1mImpMed:recnum,...,Bf1mBP:recnum,Bf1mWeigh:recnum,Bf1mLabTest:recnum,Bf24hCGMrn:recnum,Af2hCGMrn:recnum,Out,In,Test,Valid,Train
0,1001051,2021-07-24 14:50:00,2,1958,2,C1,63,0.0,0.0,0.0,...,0.0,0.0,0.0,289.0,24.0,False,True,False,False,True
1,1000215,2021-07-22 22:40:00,1,1974,2,C1,47,0.0,0.0,0.0,...,0.0,5.0,0.0,289.0,24.0,False,True,False,False,True
2,1000670,2021-08-13 19:50:00,2,1969,2,C1,52,0.0,0.0,0.0,...,0.0,0.0,0.0,289.0,24.0,False,True,False,False,True
3,1001035,2021-09-30 18:25:00,1,1975,2,C1,46,0.0,0.0,0.0,...,0.0,0.0,0.0,289.0,24.0,False,True,False,False,True
4,1002211,2021-07-11 05:05:00,1,1974,1,C1,47,0.0,0.0,0.0,...,0.0,2.0,0.0,289.0,24.0,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1001436,2021-11-02 03:30:00,2,1968,2,C1,53,0.0,0.0,0.0,...,0.0,0.0,0.0,289.0,24.0,False,True,False,False,True
80,1001911,2021-07-04 19:25:00,1,1994,2,C1,27,0.0,0.0,0.0,...,0.0,0.0,0.0,289.0,24.0,False,True,False,False,True
81,1001322,2021-09-25 09:05:00,2,1945,1,C1,76,0.0,0.0,0.0,...,0.0,0.0,0.0,289.0,24.0,False,True,False,False,True
82,1000106,2021-02-09 09:00:00,1,1993,1,C1,28,3.0,0.0,0.0,...,0.0,0.0,0.0,289.0,24.0,False,True,False,False,True


In [13]:
print(f'\ntotal---> recnum {len(df_case)}')
for i in ['In', 'Out', 'Train', 'Valid', 'Test']:
    print(i, df_case[i].mean())

print('Valid/(Train+Test):', df_case['Valid'].sum() / (df_case['Train'].sum() + df_case['Valid'].sum()))

subtype_list = ['cohort', 'Gender',
                 # 'patient_zipcode_3'
                 ]
# subtype_list = ['sex', 'a1cV0']
for subtype in subtype_list:
    for subname, df_sub in df_case.groupby(subtype):
        print(f'\n{subname}---> recnum {len(df_sub)}')
        for i in ['In', 'Out', 'Train', 'Valid', 'Test']:
            print(i, df_sub[i].mean())
        print('Valid/(Train+Test):', df_sub['Valid'].sum() / (df_sub['Train'].sum() + df_sub['Valid'].sum()))


total---> recnum 84
In 0.9285714285714286
Out 0.07142857142857142
Train 0.9404761904761905
Valid 0.05952380952380952
Test 0.0
Valid/(Train+Test): 0.05952380952380952

C1---> recnum 76
In 0.9342105263157895
Out 0.06578947368421052
Train 0.9342105263157895
Valid 0.06578947368421052
Test 0.0
Valid/(Train+Test): 0.06578947368421052

C2---> recnum 7
In 0.8571428571428571
Out 0.14285714285714285
Train 1.0
Valid 0.0
Test 0.0
Valid/(Train+Test): 0.0

C3---> recnum 1
In 1.0
Out 0.0
Train 1.0
Valid 0.0
Test 0.0
Valid/(Train+Test): 0.0

1---> recnum 38
In 0.9210526315789473
Out 0.07894736842105263
Train 0.9473684210526315
Valid 0.05263157894736842
Test 0.0
Valid/(Train+Test): 0.05263157894736842

2---> recnum 46
In 0.9347826086956522
Out 0.06521739130434782
Train 0.9347826086956522
Valid 0.06521739130434782
Test 0.0
Valid/(Train+Test): 0.06521739130434782
